In [1]:
import os
import glob # Adds wildcard-support for file exploration
import numpy as np
import matplotlib.pyplot as plt

%matplotlib

# Load functions from file "utils.py"
import utils

Using matplotlib backend: Qt5Agg


In [2]:
activities = [
    {"name": "Walking",
     "subdirectory": "walking",
     "label": 1},
    {"name": "Walking Upstairs",
     "subdirectory": "walking_upstairs",
     "label": 2},
    {"name": "Walking Downstairs",
     "subdirectory": "walking_downstairs",
     "label": 3},
    {"name": "Sitting",
     "subdirectory": "sitting",
     "label": 4},
    {"name": "Standing",
     "subdirectory": "standing",
     "label": 5},
    {"name": "Laying",
     "subdirectory": "laying",
     "label": 6}]

In [3]:
sampling_frequency=50.0

### Load dataset

In [4]:
# Please set the acticity-folder to load the data from
activity_directory = "walking_tf"

data_path = os.path.join("..", "data", "transfer_learning", activity_directory, "raw")
gyro_data_files = glob.glob(os.path.join(data_path,"*_gyro_sensor_data_*.txt"))
accel_data_files = glob.glob(os.path.join(data_path,"*_accel_sensor_data_*.txt"))

clipped_data_destination_path = os.path.join("..", "data", "transfer_learning", activity_directory, "clipped")

print(*gyro_data_files, sep="\n")
print("\n")
print(*accel_data_files, sep="\n")

../data/transfer_learning/walking_tf/raw/Walking_gyro_sensor_data_1622446732474.txt


../data/transfer_learning/walking_tf/raw/Walking_accel_sensor_data_1622446732474.txt


In [5]:
file_idx = 0

T_clip_start = 8
T_clip_end = 78

save = True

gyro_data_file = gyro_data_files[file_idx]
accel_data_file = gyro_data_file.replace("gyro", "accel")
gyro_data = np.loadtxt(gyro_data_file, delimiter=";")
accel_data = np.loadtxt(accel_data_file, delimiter=";")

gyro_data[:,0] = gyro_data[:,0] * 1e-3
accel_data[:,0] = accel_data[:,0] * 1e-3

In [6]:
gyro_data_file

'../data/transfer_learning/walking_tf/raw/Walking_gyro_sensor_data_1622446732474.txt'

In [7]:
accel_data_file

'../data/transfer_learning/walking_tf/raw/Walking_accel_sensor_data_1622446732474.txt'

### Interpolate and resample
The data from the sensors is not captured at a uniform sampling rate. Using linear interpolation, a sampling-rate of $50\ \text{Hz}$ is applied.

In [8]:
t_gyro_new, gyro_data_new = utils.resample_3d_timeseries(t=gyro_data[:,0], y=gyro_data[:,1:], f_resample=sampling_frequency)
t_accel_new , accel_data_new = utils.resample_3d_timeseries(t=accel_data[:,0], y=accel_data[:,1:], f_resample=sampling_frequency)

# Crop both data arrays to the same length.
if t_gyro_new[-1] < t_accel_new[-1]:
    T_max = t_gyro_new[-1]
    idx = t_accel_new <= T_max
    t_accel_new = t_accel_new[idx]
    accel_data_new = accel_data_new[idx,:]
else:
    T_max = t_accel_new[-1]
    idx = t_gyro_new <= T_max
    t_gyro_new = t_gyro_new[idx]
    gyro_data_new = gyro_data_new[idx,:]

# Check both lengths are now really the same.
assert len(t_gyro_new) == len(t_accel_new)

### Clipping Gyro-data

In [9]:
gyro_original_clip_start, gyro_original_clip_end, t_gyro_original_clipped, signals_gyro_original_clipped = \
    utils.clip_data(gyro_data[:,0], gyro_data[:,1:], T_clip_start, T_clip_end)

gyro_resampled_clip_start, gyro_resampled_clip_end, t_gyro_resampled_clipped, signals_gyro_resampled_clipped = \
    utils.clip_data(t_gyro_new, gyro_data_new, T_clip_start, T_clip_end)

In [10]:
if save:
    data_file_basename = os.path.basename(gyro_data_file)[:-4]
    resampled_clipped_data_filename = os.path.join(clipped_data_destination_path, data_file_basename + "_resampled_clipped.npy")
    original_clipped_data_filename = os.path.join(clipped_data_destination_path, data_file_basename + "_original_clipped.npy")
    
    print(resampled_clipped_data_filename)
    print(original_clipped_data_filename)
    
    np.save(original_clipped_data_filename, np.column_stack((t_gyro_original_clipped, signals_gyro_original_clipped)), allow_pickle=True)
    np.save(resampled_clipped_data_filename, np.column_stack((t_gyro_resampled_clipped, signals_gyro_resampled_clipped)), allow_pickle=True)

../data/transfer_learning/walking_tf/clipped/Walking_gyro_sensor_data_1622446732474_resampled_clipped.npy
../data/transfer_learning/walking_tf/clipped/Walking_gyro_sensor_data_1622446732474_original_clipped.npy


### Clipping Accelerometer-data

In [11]:
accel_original_clip_start, accel_original_clip_end, t_accel_original_clipped, signals_accel_original_clipped = \
    utils.clip_data(accel_data[:,0], accel_data[:,1:], T_clip_start, T_clip_end)

accel_resampled_clip_start, accel_resampled_clip_end, t_accel_resampled_clipped, signals_accel_resampled_clipped = \
    utils.clip_data(t_accel_new, accel_data_new, T_clip_start, T_clip_end)

In [12]:
if save:
    data_file_basename = os.path.basename(accel_data_file)[:-4]
    resampled_clipped_data_filename = os.path.join(clipped_data_destination_path, data_file_basename + "_resampled_clipped.npy")
    original_clipped_data_filename = os.path.join(clipped_data_destination_path, data_file_basename + "_original_clipped.npy")
    
    print(resampled_clipped_data_filename)
    print(original_clipped_data_filename)
    
    np.save(original_clipped_data_filename, np.column_stack((t_accel_original_clipped, signals_accel_original_clipped)), allow_pickle=True)
    np.save(resampled_clipped_data_filename, np.column_stack((t_accel_resampled_clipped, signals_accel_resampled_clipped)), allow_pickle=True)

../data/transfer_learning/walking_tf/clipped/Walking_accel_sensor_data_1622446732474_resampled_clipped.npy
../data/transfer_learning/walking_tf/clipped/Walking_accel_sensor_data_1622446732474_original_clipped.npy


In [13]:
plt.figure(figsize=(16,9))
plt.subplot(311)
plt.plot(t_accel_new, accel_data_new[:,0], label="Original")
plt.plot(t_accel_original_clipped, signals_accel_original_clipped[:,0], label="Original, clipped", linestyle="--")
plt.plot(t_accel_resampled_clipped, signals_accel_resampled_clipped[:,0], label="Resampled, clipped", linestyle="-.")
val_min = np.min(accel_data_new[:,0])
val_max = np.max(accel_data_new[:,0])
plt.vlines(t_accel_new[accel_resampled_clip_start], val_min, val_max, color="red")
plt.vlines(t_accel_new[accel_resampled_clip_end], val_min, val_max, color="red")
plt.grid(which="major")
plt.grid(which="minor", linestyle="--")
plt.minorticks_on()
plt.legend()
plt.title("Accelerometer")

plt.subplot(312)
plt.plot(t_accel_new, accel_data_new[:,1])
plt.plot(t_accel_original_clipped, signals_accel_original_clipped[:,1], label="Original, clipped", linestyle="--")
plt.plot(t_accel_resampled_clipped, signals_accel_resampled_clipped[:,1], label="Resampled, clipped", linestyle="-.")
val_min = np.min(accel_data_new[:,1])
val_max = np.max(accel_data_new[:,1])
plt.vlines(t_accel_new[accel_resampled_clip_start], val_min, val_max, color="red")
plt.vlines(t_accel_new[accel_resampled_clip_end], val_min, val_max, color="red")
plt.grid(which="major")
plt.grid(which="minor", linestyle="--")
plt.minorticks_on()
plt.legend()

plt.subplot(313)
plt.plot(t_accel_new, accel_data_new[:,2])
plt.plot(t_accel_original_clipped, signals_accel_original_clipped[:,2], label="Original, clipped", linestyle="--")
plt.plot(t_accel_resampled_clipped, signals_accel_resampled_clipped[:,2], label="Resampled, clipped", linestyle="-.")
val_min = np.min(accel_data_new[:,2])
val_max = np.max(accel_data_new[:,2])
plt.vlines(t_accel_new[accel_resampled_clip_start], val_min, val_max, color="red")
plt.vlines(t_accel_new[accel_resampled_clip_end], val_min, val_max, color="red")
plt.grid(which="major")
plt.grid(which="minor", linestyle="--")
plt.minorticks_on()
plt.legend()

In [14]:
plt.figure(figsize=(16,9))
plt.subplot(311)
plt.plot(t_gyro_new, gyro_data_new[:,0], label="Original")
plt.plot(t_gyro_original_clipped, signals_gyro_original_clipped[:,0], label="Original, clipped", linestyle="--")
plt.plot(t_gyro_resampled_clipped, signals_gyro_resampled_clipped[:,0], label="Resampled, clipped", linestyle="-.")
val_min = np.min(gyro_data_new[:,0])
val_max = np.max(gyro_data_new[:,0])
plt.vlines(t_gyro_new[gyro_resampled_clip_start], val_min, val_max, color="red")
plt.vlines(t_gyro_new[gyro_resampled_clip_end], val_min, val_max, color="red")
plt.grid(which="major")
plt.grid(which="minor", linestyle="--")
plt.minorticks_on()
plt.legend()
plt.title("Gyroscope")

plt.subplot(312)
plt.plot(t_gyro_new, gyro_data_new[:,1])
plt.plot(t_gyro_original_clipped, signals_gyro_original_clipped[:,1], label="Original, clipped", linestyle="--")
plt.plot(t_gyro_resampled_clipped, signals_gyro_resampled_clipped[:,1], label="Resampled, clipped", linestyle="-.")
val_min = np.min(gyro_data_new[:,1])
val_max = np.max(gyro_data_new[:,1])
plt.vlines(t_gyro_new[gyro_resampled_clip_start], val_min, val_max, color="red")
plt.vlines(t_gyro_new[gyro_resampled_clip_end], val_min, val_max, color="red")
plt.grid(which="major")
plt.grid(which="minor", linestyle="--")
plt.minorticks_on()
plt.legend()

plt.subplot(313)
plt.plot(t_gyro_new, gyro_data_new[:,2])
plt.plot(t_gyro_original_clipped, signals_gyro_original_clipped[:,2], label="Original, clipped", linestyle="--")
plt.plot(t_gyro_resampled_clipped, signals_gyro_resampled_clipped[:,2], label="Resampled, clipped", linestyle="-.")
val_min = np.min(gyro_data_new[:,2])
val_max = np.max(gyro_data_new[:,2])
plt.vlines(t_gyro_new[gyro_resampled_clip_start], val_min, val_max, color="red")
plt.vlines(t_gyro_new[gyro_resampled_clip_end], val_min, val_max, color="red")
plt.grid(which="major")
plt.grid(which="minor", linestyle="--")
plt.minorticks_on()
plt.legend()